In [1]:
# pip install ipywidgets python-docx

In [ ]:
''' Developed By 
Prasad Vakhare
B. Tech. IT-1'''


import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk  # Pillow is needed for image handling
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import io
import pygame
import threading
from docx import Document
import time

# Initialize recognizer and translator
recognizer = sr.Recognizer()
translator = Translator()

# Language mapping dictionary
language_codes = {
    "Hindi": "hi",
    "Marathi": "mr",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese": "zh-cn",
    "Gujarati": "gu",
    "Urdu":"ur"
    # Add more languages as needed
}

class TranslationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Barrier Free Inclusive Education Technology")

        # Load and set the background image
        self.bg_image_raw = Image.open("Classroom.jpg")
        self.bg_image_resized = self.bg_image_raw.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.Resampling.LANCZOS)
        self.bg_image = ImageTk.PhotoImage(self.bg_image_resized)
        
        self.bg_label = tk.Label(root, image=self.bg_image)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        # Load and display the MGM image at the top center
        self.mgm_image_raw = Image.open("MGM.png")
        self.mgm_image_resized = self.mgm_image_raw.resize((200, 100), Image.Resampling.LANCZOS)  # Resize MGM image
        self.mgm_image = ImageTk.PhotoImage(self.mgm_image_resized)
        
        self.mgm_label = tk.Label(root, image=self.mgm_image, bg="light blue")
        self.mgm_label.place(relx=0.5, rely=0.07, anchor='center')  # Place it at the top center

        # University Department Label with shadow effect, centered with a green border
        self.department_label = tk.Label(root, text="UNIVERSITY DEPARTMENT OF INFORMATION AND COMMUNICATION TECHNOLOGY",
                                         bg="light yellow", fg="brown", font=("Helvetica", 18, "bold"),
                                         bd=5, relief="solid")
        self.department_label.place(relx=0.5, rely=0.17, anchor='center')
        
        # Project label
        self.project_label = tk.Label(root, text="Barrier Free Inclusive Education Technology",
                                         bg="light yellow", fg="brown", font=("Helvetica", 18, "bold"),
                                         bd=5, relief="solid")
        self.project_label.place(relx=0.5, rely=0.23, anchor='center')

        # Selected language
        self.selected_language = tk.StringVar(value="Hindi")
        
        # Language selection dropdown
        self.language_label = tk.Label(root, text="Select Language:", bg="white", font=("Helvetica", 14, "bold"))
        self.language_label.place(relx=0.5, rely=0.30, anchor='center')
        
        self.language_dropdown = ttk.Combobox(root, textvariable=self.selected_language, values=list(language_codes.keys()), font=("Helvetica", 14))
        self.language_dropdown.place(relx=0.5, rely=0.35, anchor='center')

        # Microphone text button
        self.start_button = tk.Button(root, text="Start", font=("Helvetica", 14), command=self.start_recording, borderwidth=2, padx=20, pady=10)
        self.start_button.place(relx=0.5, rely=0.42, anchor='center')

        # Stop button with text
        self.stop_button = tk.Button(root, text="Stop", font=("Helvetica", 14), command=self.stop_recording, state=tk.DISABLED, borderwidth=2, padx=20, pady=10)
        self.stop_button.place(relx=0.5, rely=0.50, anchor='center')

        # Microphone status indicator
        self.mic_status_label = tk.Label(root, text="Idle", fg="red", bg="light blue", font=("Helvetica", 14))
        self.mic_status_label.place(relx=0.5, rely=0.58, anchor='center')

        # Input text box
        self.input_text_label = tk.Label(root, text="Input Audio Text:", font=("Helvetica", 14, "bold"), bg="white")
        self.input_text_label.place(relx=0.23, rely=0.26, anchor='center')

        self.input_text_box = tk.Text(root, height=17, width=46, font=("Nirmala UI", 15, "bold"), padx=10, pady=10, relief="solid", bd=2, wrap="word")
        self.input_text_box.place(relx=0.23, rely=0.61, anchor='center')
        self.input_text_box.config(bg="#E6E6FA", highlightbackground="#E6E6FA", highlightcolor="#E6E6FA")  # Using lavender color
        # Adjust line spacing using tags
        self.input_text_box.tag_configure("spacing", spacing3=10)  # Adjust the spacing3 value as needed

        # Output text box
        self.output_text_label = tk.Label(root, text="Translated Text:", font=("Helvetica", 14, "bold"), bg="white")
        self.output_text_label.place(relx=0.77, rely=0.26, anchor='center')

        self.output_text_box = tk.Text(root, height=17, width=46, font=("Nirmala UI", 15, "bold"), padx=10, pady=10, relief="solid", bd=2, wrap="word")
        self.output_text_box.place(relx=0.77, rely=0.61, anchor='center')
        self.output_text_box.config(bg="#E6E6FA", highlightbackground="#E6E6FA", highlightcolor="#E6E6FA")  # Using lavender color
        # Adjust line spacing using tags
        self.output_text_box.tag_configure("spacing", spacing3=10)  # Adjust the spacing3 value as needed
        
        # Save button
        self.save_button = tk.Button(root, text="Save", font=("Helvetica", 12), command=self.save_text, borderwidth=1, padx=20, pady=10)
        self.save_button.place(relx=0.23, rely=0.97, anchor='center')
        
        # Clear button
        self.clear_button = tk.Button(root, text="Clear", font=("Helvetica", 12), command=self.clear_text, borderwidth=1, padx=20, pady=10)
        self.clear_button.place(relx=0.77, rely=0.97, anchor='center')

        # Load and display the Anchor image at the bottom center
        self.anchor_image_raw = Image.open("Anchor2.png").convert("RGBA")
        self.anchor_image_resized = self.anchor_image_raw.resize((200, 300), Image.Resampling.LANCZOS)  # Increase the size
        self.anchor_image = ImageTk.PhotoImage(self.anchor_image_resized)
        
        self.anchor_label = tk.Label(root, image=self.anchor_image, bg="#c1b8b4")
        self.anchor_label.place(relx=0.5, rely=0.82, anchor='center')  # Place it at the bottom center

        # Store images as instance variables to prevent garbage collection
        self.images = {
            'bg_image': self.bg_image,
            'mgm_image': self.mgm_image,
            'anchor_image': self.anchor_image
        }

        # Temporary storage for audio
        self.audio_data = None
        self.recording_thread = None

    def start_recording(self):
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.audio_data = None  # Reset audio data
        self.recording_thread = threading.Thread(target=self.record_audio)
        self.recording_thread.start()

    def record_audio(self):
        try:
            self.update_mic_status("Listening...", "green")  # Update status to show listening
            with sr.Microphone() as source:
                recognizer.adjust_for_ambient_noise(source)
                self.audio_data = recognizer.listen(source)
        except Exception as e:
            self.input_text_box.insert(tk.END, f"Error: {e}\n")
            self.input_text_box.see(tk.END)
        finally:
            self.update_mic_status("Idle", "red")  # Update status back to idle
            self.root.after(0, self.stop_recording)

    def stop_recording(self):
        if self.audio_data:
            self.process_audio()
        else:
            self.input_text_box.insert(tk.END, "No audio recorded\n")
            self.input_text_box.see(tk.END)
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        
    def process_audio(self):
        try:
            # Recognize speech
            english_text = recognizer.recognize_google(self.audio_data, language='en-US')
            self.input_text_box.insert(tk.END, "Input: " + english_text + "\n")
            self.input_text_box.see(tk.END)

            # Translate text
            translated_text = translator.translate(english_text, dest=language_codes[self.selected_language.get()]).text
            self.output_text_box.insert(tk.END, "Output: " + translated_text + "\n")
            self.output_text_box.see(tk.END)
            # Convert translated text to speech
            tts = gTTS(translated_text, lang=language_codes[self.selected_language.get()])
            fp = io.BytesIO()
            tts.write_to_fp(fp)
            fp.seek(0)

            # Schedule the audio playback to run in the main thread
            self.root.after(0, self.play_audio, fp)

        except sr.UnknownValueError:
            self.input_text_box.insert(tk.END, "Could not understand the audio\n")
            self.input_text_box.see(tk.END)
        except sr.RequestError as e:
            self.input_text_box.insert(tk.END, f"Error: {e}\n")
            self.input_text_box.see(tk.END)

    def play_audio(self, audio_file):
        pygame.mixer.init()
        pygame.mixer.music.load(audio_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            self.root.update()  # Keep the UI responsive during playback
            
    def save_text(self):
        # Get text from input and output text boxes
        input_text = self.input_text_box.get("1.0", tk.END).strip()
        output_text = self.output_text_box.get("1.0", tk.END).strip()

        if input_text or output_text:
            # Create separate Word files for input and output text
            timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
            
            input_filename = f"input_text_{timestamp}.docx"
            output_filename = f"output_text_{timestamp}.docx"

            # Save input text to Word file
            input_doc = Document()
            input_doc.add_paragraph(input_text)
            input_doc.save(input_filename)

            # Save output text to Word file
            output_doc = Document()
            output_doc.add_paragraph(output_text)
            output_doc.save(output_filename)

            # Clear both text boxes
            self.clear_text()
    
    def clear_text(self):
        
        # Clear the text boxes after saving
        self.input_text_box.delete("1.0", tk.END)
        self.output_text_box.delete("1.0", tk.END)

    def update_mic_status(self, status_text, color):
        self.mic_status_label.config(text=status_text, fg=color)
        self.root.update_idletasks()

if __name__ == "__main__":
    root = tk.Tk()

    # Maximize the window to make the image fit the entire screen
    root.state('zoomed')

    app = TranslationApp(root)
    root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
